In [3]:
import gc
gc.collect()

11

In [4]:
import time
T0 = time.time()

In [5]:
import json
import numpy as np
import pandas as pd
from sklearn.cluster import SpectralClustering
import sys, os
from copy import copy
#!/usr/bin/python
# parent = os.path.dirname(os.path.realpath(__file__))
sys.path.append('~/mitie')

from mitie import *
from collections import defaultdict

ner = named_entity_extractor('../../../mitie/mitie-v0.2-python-2.7-windows-or-linux64/MITIE-models/english/ner_model.dat')

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag


In [6]:
from collections import defaultdict
pd.options.display.max_colwidth = 250

In [7]:
pd.options.display.max_colwidth = 250
pd.options.display.max_rows = 400

In [8]:
vocabdict = defaultdict(int)

In [9]:
lemmatizer = WordNetLemmatizer()
def lemmatize_me(x):
    try:
        x_tagged = pos_tag(x)
    except:
        return 'failed'
    return_list = []
    for i in x_tagged:
        if 'V' in i[1]:
            value = 'v'
        else:
            value = 'n'
        return_list.append(lemmatizer.lemmatize(i[0], pos = value))
    return ' '.join(return_list)
    

In [10]:
def remove_blanks(x):
    x_lis = x
    while True:
        try:
            x_lis.remove('')
        except:
            return x_lis

In [11]:
def date_conv(x):
    x = x.strip().replace(u'\xa0',' ').encode('utf-8')
    split = x.split('T')
    if (x.find('T') != len(x) - 1) or (x.find('T') == -1) :
        return pd.to_datetime(split[0], format='%Y-%m-%d')
    else:
        split = x.split(' ')
        date = '-'.join(split[:3])
        return pd.to_datetime(date, format='%d-%b-%Y')


In [12]:
## Data should be pre-tokenized within a dataframe, this is intended to be applied to that column of the dataframe and will output  
entities_dict = defaultdict(int)


def get_ent_simp(x, dictname=entities_dict):
        x = tokenize(x)
        entities = ner.extract_entities(x)
        
        for e in entities:
            rangee = e[0]
            entity_text = " ".join(x[i] for i in rangee)
            dictname[entity_text]+=1
        


In [13]:
def len_filter(x):
    ret_list = []
    for i in x:
        if len(i) > 4:
            ret_list.append(i)
    return ret_list

In [14]:
def clean_text(x):
    val = x.encode('utf-8').replace("\xe2\x80\x99","'").replace("\xe2\x80\x9c",'"').replace("\xe2\x80\x9d",'"').replace('\xe2\x80\x94',"'").replace('\xe2\x80\xa6','...')
    return val

In [15]:
def trim_data(x):
    if len(focus_vocab) <= 5:
        if len(set(x).intersection(set(focus_vocab))) == len(focus_vocab):
            return 1
    elif len(set(x).intersection(set(focus_vocab))) >  round(len(focus_vocab)*0.6):
            return 1
    else:
           return 0

In [16]:
"""
Visualization of matrix block structure
(eg, pairwise similarity or co-occurrence)
Requires scaledimage.py for intensity plots
David Andrzejewski
"""
import numpy as NP
import matplotlib.pyplot as P
import matplotlib.lines as L
import sklearn.cluster as SKLC

# from scaledimage import scaledimage

def blockviz(affinity, nclust, ax=None):


    # Do spectral clustering
    ndata = affinity.shape[0]
    c = SKLC.SpectralClustering(n_clusters=nclust, affinity='linear', random_state=1445)
    c.fit(affinity)
    # Extract cluster labels and sort indices to align with clusters
    sortidx = []
    for ki in range(nclust):
        sortidx += getlabeled(c.labels_, ki)
    sorted_affinity = affinity.copy()
    sorted_affinity = sorted_affinity[sortidx,:]
    sorted_affinity = sorted_affinity[:,sortidx]
    kstart = 0
    clus_size = []
    clus_start = []
    for ki in range(nclust):    
        clustki = getlabeled(c.labels_, ki)
        clus_size.append(clustki)
        kstart += len(clustki)
        clus_start.append(kstart)
    return clus_size,clus_start, sorted_affinity,c.labels_

def logistic(val):
    """ Logistic function """
    return float(1) / (1 + NP.exp(-1 * val))

def getlabeled(labels, ki):
    """ Get indices where labels==ki """
    return [idx for (idx, val) in 
            enumerate(labels) if val == ki]

def drawH(ax, y, xstart, xend):
    """ Draw horiztonal line """
    ax.add_line(L.Line2D([xstart, xend], 
                  [y, y],
                  color='r'))

def drawV(ax, x, ystart, yend):
    """ Draw vertical line """
    ax.add_line(L.Line2D([x, x],
                         [ystart, yend], 
                         color='r'))

def drawClust(ax, kstart, kend, kmax, scale=1):
    """ Draw bounding box for cluster """
    skstart = scale * kstart
    skend = scale * kend
    skmax = scale * kmax
    if(skstart == 0):
        # Upper-left cluster: only draw bottom-right borders
        drawH(ax, skmax-skend, skstart, skend)
        drawV(ax, skend, skmax-skstart, skmax-skend)
    elif(skend == skmax):
        # Lower-right cluster: only draw top-left borders
        drawH(ax, skmax-skstart, skstart, skend)
        drawV(ax, skstart, skmax-skstart, skmax-skend)
    else:
        # Otherwise, draw all 4 borders
        drawH(ax, skmax-skend, skstart, skend)
        drawV(ax, skend, skmax-skstart, skmax-skend)
        drawH(ax, skmax-skstart, skstart, skend)
        drawV(ax, skstart, skmax-skstart, skmax-skend)
        
import numpy as NP
import matplotlib.pyplot as P
import matplotlib.ticker as MT
import matplotlib.cm as CM
 
def scaledimage(W, pixwidth=20, ax=None, grayscale=True):
    """
    Do intensity plot, similar to MATLAB imagesc()
 
    W = intensity matrix to visualize
    pixwidth = size of each W element
    ax = matplotlib Axes to draw on 
    grayscale = use grayscale color map
 
    Rely on caller to .show()
    """
    # N = rows, M = column
    (N, M) = W.shape 
    # Need to create a new Axes?
    if(ax == None):
        ax = P.figure(figsize=(20,20)).gca()
    # extents = Left Right Bottom Top
    exts = (0, pixwidth * M, 0, pixwidth * N)
    if(grayscale):
        ax.imshow(W,
                  interpolation='nearest',
                  cmap=CM.gray,
                  extent=exts)
    else:
        ax.imshow(W,
                  interpolation='nearest',
                  extent=exts)
 
    ax.xaxis.set_major_locator(MT.NullLocator())
    ax.yaxis.set_major_locator(MT.NullLocator())
    return ax
 
if __name__ == '__main__':
    # Define a synthetic test dataset
     testweights = NP.array([[0.25, 0.50, 0.25, 0.00],
                            [0.00, 0.50, 0.00, 0.00],
                            [0.00, 0.10, 0.10, 0.00],
                            [0.00, 0.00, 0.25, 0.75]])
    # Display it
#     ax = scaledimage(testweights)
#     P.show()


### Open and read file

In [17]:
files = ['161208_alja.json','161208_econ.json','161208_guard.json','161208_nyt.json','161208_wsj.json']
splits = []
for file_ in files:
    f = open(file_,'r')
    filestring = f.read()
    split = filestring.replace(']\n','').replace('[\n','').replace('\n]','').split(',\n')
    splits.extend(split)
final_dict = defaultdict(tuple)

In [18]:
 splits[0]

'{"url": "http://www.aljazeera.com/news/2016/12/iraqi-army-denies-bombing-civilians-isil-held-qaim-161208153959672.html", "date": "08 Dec 2016 18:55 GMT", "ref_url": "http://www.aljazeera.com", "title": "Iraqi army denies bombing civilians in ISIL-held Qaim", "text": " Iraq\'s military responded to what it called a \\"fake story\\" that alleged its air strikes mistakenly killed dozens of civilians in the ISIL-held city of al-Qaim, accusing media and politicians of doing propaganda work for the armed group. Iraq\'s Joint Operations Command said on Thursday the air raids a day earlier targeted dozens of foreign fighters of Islamic State of Iraq and the Levant (ISIL, also known as ISIS) - not civilians. READ MORE: Iraqi army mistakenly bombs civilians in ISIL-held city It said the first of two air strikes was conducted at 09:00 GMT and struck a two-storey building housing 25 mostly foreign would-be suicide bombers, led by a fighter it named as Abu Maysar al-Kawkazi. Another strike was car

In [19]:
fail_count = 0
for i in splits:
    try:
        i = json.loads(i)
        url = i['url']
        itup = (i['date'], i['text'], i['title'])
        final_dict[url] = itup
    except:
        fail_count +=1
        continue
    
    

In [20]:
 print fail_count

0


In [21]:
 len(final_dict.keys())

90804

In [22]:
 fin_json = json.dumps(final_dict)


In [23]:
 f = open('final_article.json', 'w')

In [24]:
#  f.write(fin_json)

In [25]:
f.close()

### Create dataframe

In [26]:
df = pd.read_json(fin_json, orient='index')
df = df.reset_index()
df.columns = ['url','date','text','title']

In [27]:
df.shape

(90804, 4)

In [29]:
df['len'] = df['text'].apply(lambda x: len(x))
df = df[df['len'] > 100]

In [30]:
df = df.reset_index().drop('index', axis=1)

In [31]:
df.shape

(90804, 5)

In [32]:

# df['date'] = df['url'].apply(lambda x: x.split('.com/')[1].split('/')[:3])
# df['date'] = df['date'].apply(lambda x: '-'.join(x))
# df['length'] = df['text'].apply(lambda x: len(x))
df['tokens'] = df['text'].apply(lambda x: tokenize(x))
df['tokens'] = df['tokens'].apply(lambda x: [i.decode("utf-8").encode("ascii","ignore") for i in x])
df['tokens'] = df['tokens'].apply(lambda x: remove_blanks(x))
df['lemmas'] = df['tokens'].apply(lambda x: lemmatize_me(x))
df['source'] = df['url'].apply(lambda x: x.split('.')[1])
df = df.reset_index().drop('index',axis=1)

In [33]:
df['source'].value_counts()

nytimes        39237
theguardian    27055
aljazeera      10564
economist       8840
wsj             5108
Name: source, dtype: int64

In [34]:
df['datetime'] = df['date'].apply(lambda x: date_conv(x))
df['year'] = df['datetime'].apply(lambda x: x.year)
df['month'] = df['datetime'].apply(lambda x: x.month)
df['tlower'] = df['text'].apply(lambda x: tokenize(x.lower()))

In [35]:
df.to_json('df_titled.json', orient='values')

### Restart Here, load dataframe from JSON

In [ ]:
# df = pd.read_json('/journal/df_final.json', orient='values')


In [ ]:
# df.columns = ['url','date','text','len','tokens','lemmas','source','datetime','year','month']


In [ ]:
# df['tlower'] = df['text'].apply(lambda x: tokenize(x.lower()))

## trim to single month

In [36]:
years = [2012,2013,2014,2015,2016]
months = [1,2,3,4,5,6,7,8,9,10,11,12]

In [37]:
for year in years:
    for month in months:
        df_month = copy(df[(df['year'] == year) & (df['month'] == month)])
        cv2 = CountVectorizer(stop_words='english',max_df = 0.2, min_df = 0.01, strip_accents = 'unicode',analyzer='word').fit(df_month['lemmas'])
        test = cv2.vocabulary_.keys()
        for i in test: 
            if (len(i) <=4) or (any(char.isdigit() for char in i)): test.pop(test.index(i))
        unfiltered_vocab = pos_tag(test)
        final_vocab = []
        for i in unfiltered_vocab:
            if (i[1] in ('NN','NNS')) & (len(i[0]) > 4):
                final_vocab.append(i[0])
        entities_dict = defaultdict(int)
        df_month['text'].apply(lambda x: get_ent_simp(x))
        entities_set = set()
        for k,v in entities_dict.iteritems():
            if round(len(df_month)*0.01) <= v <= round(len(df_month)*0.15):
                if len(k) > 4:
                    entities_set.update([k])    
        entities_list = [i.decode("utf-8").encode("ascii","ignore") for i in entities_set]
        entities_list = remove_blanks(entities_list)
        entities_list.extend(final_vocab)
        entities_list = list(set(entities_list))
        entities_listf = []
        [entities_listf.extend(i) for i in [x.split() for x in entities_list]];
        final_vocab = [i.decode("utf-8").encode("ascii","ignore") for i in entities_listf]
        final_vocab = list(set(final_vocab))
        final_vocab = len_filter(final_vocab)
        tv = TfidfVectorizer( stop_words='english',vocabulary=final_vocab, ngram_range=(1,5), analyzer='word').fit(df_month['lemmas'])
        tfidf = tv.transform(df_month['lemmas'])
        spec = SpectralClustering(n_clusters=25, affinity='cosine', n_init=20, assign_labels='kmeans', n_jobs=-1) 
        spec_fit = spec.fit(tfidf)
        af_mat = spec_fit.affinity_matrix_
        cluster_size, cluster_start, sorted_affinity,clusters = blockviz(af_mat, 25)
        sorted_affinity = pd.DataFrame(sorted_affinity)
        location = [0,0]
        counter = 0
        scores = []
        for i in cluster_start:
            location[1] = i-1
            score = sorted_affinity.ix[location[0]:location[1],location[0]:location[1]].mean().mean()
            scores.append(score)
            location[0] = location[1]
        df_month['spec'] = clusters
        cluster_scores = zip(xrange(0,len(scores)), scores)
        good_clusters = []
        for cluster in cluster_scores:
            if (cluster[1] >= .30) & (cluster[1] < .6):
                good_clusters.append(cluster[0])
        for cluster in good_clusters:
            filename = str(year)+ str(month)+ str(cluster)
            path = 'autoclusters/' + filename
            df_month[df_month['spec']==cluster].to_json(path, orient='values')

/home/ec2-user/anaconda2/lib/python2.7/site-packages/sklearn/cluster/spectral.py:433: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn("The spectral clustering API has changed. ``fit``"
/home/ec2-user/anaconda2/lib/python2.7/site-packages/sklearn/manifold/spectral_embedding_.py:229: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


## OK, clustering part 2

In [39]:
focus

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,http://www.aljazeera.com/indepth/features/2013/05/201352071040948543.html,20 May 2013 13:08 GMT,"Mes Aynak, Afghanistan - Ruins dot each hilltop in mountainous Mes Aynak, an hour's drive south of the capital Kabul. Buddhist monasteries stood here for hundreds of years, and Afghan workers under the supervision of archaeologists are racing to...",Afghan archaeology site faces rocky future,5985,"[Mes, Aynak, ,, Afghanistan, -Ruins, dot, each, hilltop, in, mountainous, Mes, Aynak, ,, an, hour, 's, drive, south, of, the, capital, Kabul, ., Buddhist, monasteries, stood, here, for, hundreds, of, years, ,, and, Afghan, workers, under, the, su...","Mes Aynak , Afghanistan -Ruins dot each hilltop in mountainous Mes Aynak , an hour 's drive south of the capital Kabul . Buddhist monastery stand here for hundred of year , and Afghan worker under the supervision of archaeologist be race to uncov...",aljazeera,1369008000000,2013,5,"[mes, aynak, ,, afghanistan, - ruins, dot, each, hilltop, in, mountainous, mes, aynak, ,, an, hour, 's, drive, south, of, the, capital, kabul, ., buddhist, monasteries, stood, here, for, hundreds, of, years, ,, and, afghan, workers, under, the, s...",12
1,http://www.nytimes.com/2013/05/10/world/asia/karzai-says-us-can-keep-afghan-bases-after-2014.html,2013-05-09T12:38:34-04:00,"KABUL, Afghanistan — Afghanistan is ready to let the United States and its allies keep military bases here after the end of the NATO combat mission next year, President Hamid Karzai said on Thursday, offering a concrete public signal that foreig...","Karzai Says U.S. Bases Can Stay, Raising Some Eyebrows in West",5166,"[KABUL, ,, Afghanistan, Afghanistan, is, ready, to, let, the, United, States, and, its, allies, keep, military, bases, here, after, the, end, of, the, NATO, combat, mission, next, year, ,, President, Hamid, Karzai, said, on, Thursday, ,, offering...","KABUL , Afghanistan Afghanistan be ready to let the United States and it ally keep military base here after the end of the NATO combat mission next year , President Hamid Karzai say on Thursday , offer a concrete public signal that foreign troop ...",nytimes,1368057600000,2013,5,"[kabul, ,, afghanistan, —, afghanistan, is, ready, to, let, the, united, states, and, its, allies, keep, military, bases, here, after, the, end, of, the, nato, combat, mission, next, year, ,, president, hamid, karzai, said, on, thursday, ,, offer...",12
2,http://www.nytimes.com/2013/05/12/world/asia/afghanistan-and-us-continue-talks-on-future-relations.html,2013-05-11T14:33:35-04:00,"KABUL, Afghanistan (AP) — Senior American and Afghan officials held talks on Saturday to try to iron out the details of a pact that defines the future of America’s commitment to Afghanistan. The strategic partnership agreement outlines a set of ...",U.S. and Afghans Negotiate Future,2231,"[KABUL, ,, Afghanistan, (, AP, ), Senior, American, and, Afghan, officials, held, talks, on, Saturday, to, try, to, iron, out, the, details, of, a, pact, that, defines, the, future, of, America, 's, commitment, to, Afghanistan, ., The, strategic,...","KABUL , Afghanistan ( AP ) Senior American and Afghan official hold talk on Saturday to try to iron out the detail of a pact that define the future of America 's commitment to Afghanistan . The strategic partnership agreement outline a set of pri...",nytimes,1368230400000,2013,5,"[kabul, ,, afghanistan, (, ap, ), —, senior, american, and, afghan, officials, held, talks, on, saturday, to, try, to, iron, out, the, details, of, a, pact, that, defines, the, future, of, america, 's, commitment, to, afghanistan, ., the, strateg...",12
3,http://www.nytimes.com/2013/05/13/world/asia/afghans-say-an-american-tortured-civilians.html,2013-05-12T20:05:42-04:00,"MAIDAN SHAHR, Afghanistan — The authorities in Afghanistan are seeking the arrest on murder and torture charges of a man they say is an American and part of a Special Forces unit operating in Wardak Province, three A

In [44]:
for filename in os.listdir('./autoclusters'):
    directory = 'autoclusters/'
    file_loc = directory + filename
    focus = pd.read_json(file_loc, orient='values')
    focus.columns = ['url','date','text','title','length','tokens','lemmas','source','datetime','year','month','tlower','cluster']
    cv_focus = CountVectorizer(stop_words='english', min_df = 0.7, ngram_range=(1,5),  strip_accents = 'unicode',analyzer='word').fit(focus['lemmas'])
    focus_vocab = set(cv_focus.vocabulary_.keys())
    focus_vocab = len_filter(focus_vocab)
    focus_vocab = list(set(focus_vocab))
    focus_vocab = len_filter(focus_vocab)
    try:
        focus_vocab.pop(focus_vocab.index('Mr'))
    except:
        pass    
    df['filter'] = df['tlower'].apply(lambda x: trim_data(x))
    df_trim = copy(df[df['filter']==1])
    if (len(df_trim) > 2*len(focus)) & (len(df_trim)>100) & (len(df_trim) < 20000):

        tfidf_focus = TfidfVectorizer( stop_words='english',vocabulary=focus_vocab, ngram_range=(1,3), analyzer='word').fit(df_trim['lemmas'])
        tff = tfidf_focus.transform(df_trim['lemmas'])
        tff = tff.todense()
        nclusters = max(min((len(focus_vocab)-1), len(df_trim)/2),5)
        spec_focus = SpectralClustering(n_clusters= nclusters, affinity='cosine', n_init=25, assign_labels='kmeans', n_jobs = -1)
        sf_fit = spec_focus.fit(tff)
        af_mat = sf_fit.affinity_matrix_
        cluster_size, cluster_start, sorted_affinity,clusters = blockviz(af_mat, nclusters)
        sorted_affinity = pd.DataFrame(sorted_affinity)
        location = [0,0]
        counter = 0
        scores = []
        for i in cluster_start:
            location[1] = i-1
            score = sorted_affinity.ix[location[0]:location[1],location[0]:location[1]].mean().mean()
            scores.append(score)
            location[0] = location[1]
        df_trim['spec'] = clusters
        cluster_scores = zip(xrange(0,len(scores)), scores)
        good_clusters = []
        for cluster in cluster_scores:
            if (cluster[1] >= .85) & (cluster[1] < .9):
                if len(df_trim[df_trim['spec']==cluster[0]]) > len(focus):
                    good_clusters.append(cluster)

        for cluster in good_clusters:
                outname = str(filename)+'_'+ str(cluster[1])+'_'+str(cluster[0])
                path = 'refined/' + outname
                df_trim[df_trim['spec']==cluster[0]].to_json(path, orient='values')

        if good_clusters == []:
            if len(focus) > 25:
                outname = str(filename)+'_nothingbetter'
                path = 'good_clusters/' + outname
                focus.to_json(path, orient='values')



LinAlgError: the leading minor of order 14 of 'b' is not positive definite. The factorization of 'b' could not be completed and no eigenvalues or eigenvectors were computed.

In [45]:
cluster

(3, 0.8944357816397399)

In [46]:
len(focus_vocab)

2

In [47]:
len(df_trim)

1229

In [48]:
len(focus)

8

In [49]:
max(min((len(focus_vocab)-1), len(df_trim)/2),5)

5

In [51]:
df_trim.head()

,url,date,text,title,len,tokens,lemmas,source,datetime,year,month,tlower,filter
3,http://www.aljazeera.com/blogs/2015/01/obama-visit-india-rhetoric-actions-150127135830901.html,27 Jan 2015 14:17 GMT,"With Obama's plane taking off from Palam Airport for Riyadh, Saudi Arabia, it's time to look beyond the rhetoric. Two great countries, two great democracies, two great leaders, the security, the hospitality, the bromance, the chewing gum and the...",Obama's visit to India: Rhetoric vs actions,3430,"[With, Obama, 's, plane, taking, off, from, Palam, Airport, for, Riyadh, ,, Saudi, Arabia, ,, it, 's, time, to, look, beyond, the, rhetoric, ., Two, great, countries, ,, two, great, democracies, ,, two, great, leaders, ,, the, security, ,, the, h...","With Obama 's plane take off from Palam Airport for Riyadh , Saudi Arabia , it 's time to look beyond the rhetoric . Two great country , two great democracy , two great leader , the security , the hospitality , the bromance , the chewing gum and ...",aljazeera,2015-01-27,2015.0,1.0,"[with, obama, 's, plane, taking, off, from, palam, airport, for, riyadh, ,, saudi, arabia, ,, it, 's, time, to, look, beyond, the, rhetoric, ., two, great, countries, ,, two, great, democracies, ,, two, great, leaders, ,, the, security, ,, the, h...",1.0
126,http://www.aljazeera.com/blogs/americas/2015/01/obama-finds-modi-special-150123211300109.html,23 Jan 2015 21:24 GMT,The Obama Administration is doing something it doesn’t usually do. As the president gets ready to fly to India they are stressing the bond he shares with Prime Minister Narendra Modi. This was how one of the president’s top adviser’s Ben Rhodes...,Why Obama finds Modi so special,3333,"[The, Obama, Administration, is, doing, something, it, doesn, 't, usually, do, ., As, the, president, gets, ready, to, fly, to, India, they, are, stressing, the, bond, he, shares, with, Prime, Minister, Narendra, Modi, ., This, was, how, one, of,...",The Obama Administration be do something it doesn 't usually do . As the president get ready to fly to India they be stress the bond he share with Prime Minister Narendra Modi . This be how one of the president 's top adviser 's Ben Rhodes recent...,aljazeera,2015-01-23,2015.0,1.0,"[the, obama, administration, is, doing, something, it, doesn, 't, usually, do, ., as, the, president, gets, ready, to, fly, to, india, they, are, stressing, the, bond, he, shares, with, prime, minister, narendra, modi, ., this, was, how, one, of,...",1.0
262,http://www.aljazeera.com/blogs/asia/2011/04/23676.html,06 Apr 2011 12:52 GMT,"India is still in a state of euphoria following their World Cup win. &nbsp It was the prefect end to a tournament for both the fans and the organisers, mindful of the TV ratings disaster that accompanied India's early exit from the tournament in...",Cricket World Cup - final thoughts,3081,"[India, is, still, in, a, state, of, euphoria, following, their, World, Cup, win, ., &nbsp, It, was, the, prefect, end, to, a, tournament, for, both, the, fans, and, the, organisers, ,, mindful, of, the, TV, ratings, disaster, that, accompanied, ...","India be still in a state of euphoria follow their World Cup win . &nbsp It be the prefect end to a tournament for both the fan and the organiser , mindful of the TV rating disaster that accompany India 's early exit from the tournament in 2007 a...",aljazeera,2011-04-06,2011.0,4.0,"[india, is, still, in, a, state, of, euphoria, following, their, world, cup, win, ., &nbsp, it, was, the, prefect, end, to, a, tournament, for, both, the, fans, and, the, organisers, ,, mindful, of, the, tv, ratings, disaster, that, accompanied, ...",1.0
263,http://www.aljazeera.com/blogs/asia/2011/04/24396.html,08 Apr 2011 21:34 GMT,"It is being called India's version of Tahrir Square after Egypt's protests that toppled the president. Jantar Mantar, the country's historic stretch, is crowded with Indians. They're singing, chanting, dancing, painting, holding up placards in 4...",Will Indian protes

In [ ]:
min(10,10,23,4,1214)

In [ ]:
df_trim[df_trim['spec'] == 2]['url']

In [ ]:
# entities_dict = defaultdict(int)
#     focus['text'].apply(lambda x: get_ent_simp(x, entities_dict));
#     entities_set = set()
#     for k,v in entities_dict.iteritems():
#         if round(len(focus))*0.5 <= v <= round(len(focus)*10):
#             if len(k) > 4:
#                 entities_set.update([k])
#     entities_list = []
#     entities_list = [i.decode("utf-8").encode("ascii","ignore") for i in entities_set]            
#     entities_list = remove_blanks(entities_list)
#     entities_list = list(set(entities_list))
#     entities_listf = []
#     [entities_listf.extend(i) for i in [x.split() for x in entities_list]];
#     focus_vocab.extend(entities_listf)

In [ ]:
focus.columns = ['url','date','text','length','tokens','lemmas','source','datetime','year','month','cluster']

In [ ]:
location = [0,0]
counter = 0
scores = []
for i in cluster_start:
    location[1] = i-1
    score = sorted_affinity.ix[location[0]:location[1],location[0]:location[1]].mean()
    scores.append(score)
    location[0] = location[1]

In [ ]:
location[1] = 291

In [ ]:
df['filter'] = df['tokens'].apply(lambda x: trim_data(x))

In [ ]:
df_trim = copy(df[df['filter']==1])

In [ ]:
entities_set = set()
for k,v in entities_dict.iteritems():
    if round(len(focus))*0.5 <= v <= round(len(focus)*10):
        if len(k) > 4:
            entities_set.update([k])
entities_list = []
entities_list = [i.decode("utf-8").encode("ascii","ignore") for i in entities_set]            
entities_list = remove_blanks(entities_list)
entities_list = list(set(entities_list))
entities_listf = []
[entities_listf.extend(i) for i in [x.split() for x in entities_list]];

In [ ]:
entities_dict

In [ ]:
focus_vocab.extend(entities_listf)

In [ ]:
focus_vocab = list(set(focus_vocab))

In [ ]:
focus_vocab.pop(focus_vocab.index('Mr'))

In [ ]:
focus_vocab

In [ ]:
t0 = time.time()
tfidf_focus = TfidfVectorizer( stop_words='english',vocabulary=focus_vocab, ngram_range=(1,3), analyzer='word').fit(df_trim['lemmas'])
tff = tfidf_focus.transform(df_trim['lemmas'])
tff
t1 = time.time()

In [ ]:
tff

In [ ]:
tff = tff.todense()

In [ ]:
t1-t0

In [ ]:
tff.shape

In [ ]:
spec_focus = SpectralClustering(n_clusters=15, affinity='cosine', n_init=25, assign_labels='kmeans', n_jobs = -1)

In [ ]:
sf_fit = spec_focus.fit_predict(tff)

In [ ]:
tff

In [ ]:
df_trim['focus'] = sf_fit

In [ ]:
fdf = copy(df[df['focus'] == 13])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
fdf['source'].plot.hist()

In [ ]:
source_hist= df['source'].value_counts()

In [ ]:
source_hist

In [ ]:
source_df = pd.DataFrame(source_hist)

In [ ]:
source_df = source_df.transpose()

In [ ]:
source_df.plot.bar()
plt.xlabel()

In [ ]:
df_clus2 = copy(df[df['election'] == 10])

In [ ]:
tfidf_focus = TfidfVectorizer( stop_words='english',vocabulary=focus_vocab, ngram_range=(1,3), analyzer='word').fit(df_clus2['lemmas'])
tff = tfidf_focus.transform(df_clus2['lemmas'])

In [ ]:
final_clustering = SpectralClustering(n_clusters=5, n_jobs=-1, affinity='cosine', n_init=25, assign_labels='kmeans')
final_groups = final_clustering.fit_predict(tff)

In [ ]:
df_clus2['final']= final_groups

In [ ]:
df_clus2.final.value_counts()

In [ ]:
df_clus2[df_clus2['final'] == 0]['url']

In [ ]:
df_clus2.to_json('clusters/refugee_context.json')

In [ ]:
df = df.drop('refugee', axis=1)

In [ ]:
del af_mat

In [ ]:
del df_write

In [ ]:
del df_failed

In [ ]:
del split
del splits


In [ ]:
del df_worked

In [ ]:
del entities_dict, entities_list, entities_listf, entities_set

In [ ]:
del filestring


In [ ]:
del final_dict

In [ ]:
del unfiltered_vocab, vocabdict

In [ ]:
nmf_comps = NMF(n_components = 100)

In [ ]:
nmf_fit = nmf_comps.fit(tfidf)

In [ ]:
nmf_transform = nmf_fit.transform(tfidf)

In [ ]:
nmf_transform

In [ ]:
years = [2012]
months = [1]
for year in years:
    for month in months:
        df_month = copy(df[(df['year'] == year) & (df['month'] == month)])
        cv2 = CountVectorizer(stop_words='english',max_df = 0.2, min_df = 0.01, strip_accents = 'unicode',analyzer='word').fit(df_month['lemmas'])
        test = cv2.vocabulary_.keys()
        for i in test: 
            if (len(i) <=4) or (any(char.isdigit() for char in i)): test.pop(test.index(i))
        unfiltered_vocab = pos_tag(test)
        final_vocab = []
        for i in unfiltered_vocab:
            if (i[1] in ('NN','NNS')) & (len(i[0]) > 4):
                final_vocab.append(i[0])
        entities_dict = defaultdict(int)
        df_month['text'].apply(lambda x: get_ent_simp(x))
        entities_set = set()
        for k,v in entities_dict.iteritems():
            if round(len(df_month)*0.01) <= v <= round(len(df_month)*0.15):
                if len(k) > 4:
                    entities_set.update([k])    
        entities_list = [i.decode("utf-8").encode("ascii","ignore") for i in entities_set]
        entities_list = remove_blanks(entities_list)
        entities_list.extend(final_vocab)
        entities_list = list(set(entities_list))
        entities_listf = []
        [entities_listf.extend(i) for i in [x.split() for x in entities_list]];
        final_vocab = [i.decode("utf-8").encode("ascii","ignore") for i in entities_listf]
        final_vocab = list(set(final_vocab))
        final_vocab = len_filter(final_vocab)
        tv = TfidfVectorizer( stop_words='english',vocabulary=final_vocab, ngram_range=(1,5), analyzer='word').fit(df_month['lemmas'])
        tfidf = tv.transform(df_month['lemmas'])
        spec = SpectralClustering(n_clusters=25, affinity='cosine', n_init=20, assign_labels='kmeans', n_jobs=-1) 
        spec_fit = spec.fit(tfidf)
        af_mat = spec_fit.affinity_matrix_
        cluster_size, cluster_start, sorted_affinity,clusters = blockviz(af_mat, 25)
        sorted_affinity = pd.DataFrame(sorted_affinity)
        location = [0,0]
        counter = 0
        scores = []
        for i in cluster_start:
            location[1] = i-1
            score = sorted_affinity.ix[location[0]:location[1],location[0]:location[1]].mean().mean()
            scores.append(score)
            location[0] = location[1]
        df_month['spec'] = clusters
        cluster_scores = zip(xrange(0,len(scores)), scores)
        good_clusters = []
        for cluster in cluster_scores:
            if cluster[1] >= .30:
                good_clusters.append(cluster[0])


In [ ]:
cluster_scores

In [ ]:
set(focus_vocab).intersection(set(df['tokens'][2]))

In [ ]:
df[(df['source'] == 'nytimes') & (df['year'] == 2016) & (df['month']==11)]['url'][:1000]

In [ ]:
set(df.iloc[55169]['tokens']).intersection(focus_vocab)

In [ ]:
set(tokenize(df.iloc[55169]['text'].lower())).intersection(focus_vocab)

In [423]:
shapes = []
for filename in os.listdir('refined/'):
    directory = 'refined/'
    file_loc = directory + filename
    ref = pd.read_json(file_loc, orient='values')
    print filename, len(ref)
#     try: 
#         ref.columns = ['url','date','text','title','length','tokens','lemmas','source','datetime','year','month','tshort','cluster']
#         print filename, ref.shape
#         shapes.append(len(ref))
#     except:
#         continue

20161017_0.854925227791_0 462
2014914_0.867348746541_3 115
2012817_0.857823228456_2 278
201369_0.876800446215_4 121
2015115_0.887847628926_3 125
201483_0.856898964018_3 156
201550_0.867353796742_6 418
201589_0.874460711186_1 357
20151112_0.852619072971_2 96
2012212_0.875130652278_0 80
201225_0.874497302625_3 66
2015921_0.889312561455_1 121
2015713_0.869490242596_1 368
2013923_0.85270764699_2 137
20141111_0.863949060139_5 372
2015620_0.857974306639_1 52
2016621_0.860502480572_2 46
20151112_0.885640527013_0 105
2015123_0.86499726492_1 147
20151110_0.851589584328_5 172
2013105_0.889653955894_1 226
20151117_0.872713338606_0 71
201274_0.855487793009_10 147
2015108_0.862314958445_3 312
2016912_0.872761681001_0 74
2015615_0.898982075108_0 54
2014914_0.870578466479_2 195
2016111_0.864964236906_8 285
201550_0.858265522258_0 404
201457_0.892832241858_4 280
20151117_0.868103909337_4 174
2012310_0.863418023586_5 98
2015611_0.856851864377_3 68
2012522_0.883146929117_5 434
201483_0.894607405407_0 25

IOError: [Errno 21] Is a directory: 'refined/test'

In [483]:
check_df = pd.read_json('refined/2014122_0.863600010955_7', orient='values')
check_df.columns = ['url','date','text','title','length','tokens','lemmas','source','datetime','year','month','tshort','filter','cluster']
# try:
#     check_df.columns = ['url','date','text','title','length','tokens','lemmas','source','datetime','year','month','tshort','cluster']
#     print "success"
# except:
#     print 'old format'
    

In [484]:
check_df['title'].apply(lambda x: x.strip())

0                                                   Crimea poll leaves pro-Russians celebrating
1                                                     Russia's independent media face crackdown
2                                                           Sifting through the chaos of Crimea
3                                                   Tense standoff continues in eastern Ukraine
4                                       Poisoned spy inquiry reignites British-Russian tensions
5                                                                      Ukraine's other Russians
6                                                  Analysis: How to solve a problem like Syria?
7                                                        Russia vows military support for Syria
8                                             Kazakhstan jails pro-Russian 'separatist' blogger
9                                                  Putin orders troops near Ukraine to withdraw
10                                      

In [495]:
refined_list = ['2015713_0.869490242596_1','201483_0.894607405407_0',
                '2015108_0.862314958445_3','2012310_0.863418023586_5',
                '20151117_0.868103909337_4','2014914_0.867348746541_3','201369_0.876800446215_4',
                '201550_0.867353796742_6','201589_0.874460711186_1',
                '2015921_0.889312561455_1','20141111_0.863949060139_5',
                '2015620_0.857974306639_1','2015123_0.86499726492_1','20151110_0.851589584328_5',
                '2013105_0.889653955894_1','20151117_0.872713338606_0','201274_0.855487793009_10',
                '2016912_0.872761681001_0','2015615_0.898982075108_0','2014914_0.870578466479_2',
                '2016111_0.864964236906_8','2016621_0.880303583745_0','2015915_0.853669332983_0',
                '201274_0.869142411295_11','2014816_0.888438440359_2','2014122_0.863600010955_7',
                '2015519_0.854170619314_0','201615_0.894225562833_1','2013916_0.865737531586_7',
                '2015519_0.858457904117_3','2016720_0.868183998473_0','20141111_0.875494072805_7',
                '20141111_0.877464800424_1', '201568_0.882025726485_2', '2016712_0.878390449168_3',
                '201267_0.855970183428_1','20121211_0.85047660526_4','2015109_0.889486345068_3',
                '2015108_0.869122322263_1', '2014921_0.873457519386_2','2013417_0.85096034015_2',
                '20131112_0.882619349327_9','2016713_0.861363052328_2','201274_0.853896994832_0',
                '201638_0.857744937876_15', '201435_0.884174377166_0','201615_0.858266502028_0',
                '2015620_0.890818713208_0'
                
               ]

refined_tags = ['Election 2016','Israel & Palestine', 
                'Election 2016','Supreme Court',
                'Israel & Palestine', 'Climate Change','Egyptian Revolution',
                'Brexit','Election 2016',
                'Apple Inc.','Russian Foreign Policy',
               'Yemeni Civil War','Climate Change','Election 2016',
               'Climate Change','Israel & Palestine','Syrian Civil War',
                'Philippines Drug War','FIFA Corruption','Climate Change',
                'Election 2016','Indian Politics','UK Politics',
                'Syrian Civil War','Indian Politics','Russian Foreign Policy',
                'Saudi Foreign Policy','Brexit','Egyptian Revolution',
                'Yemeni Civil War','Israel & Palestine','Russian Foreign Policy',
                'Russian Foreign Policy','Election 2016', 'Russian Foreign Policy',
                'Syrian Civil War','Israel','VW Scandal', 
                'Election 2016', 'Police Violence','Brexit',
                'Climate Change','Election 2016','Syrian Civil War',
                'Election 2016','Virtual Reality','Brexit',
                'Saudi Foreign Policy'
               ]


In [496]:
good_clusters = ['201633_nothingbetter','2016413_nothingbetter','2014610_nothingbetter',
                '20161215_nothingbetter','201634_nothingbetter','20151011_nothingbetter','201585_nothingbetter','201644_nothingbetter',
                '201690_nothingbetter','201593_nothingbetter', '2015103_nothingbetter','201623_nothingbetter']

good_tags = ['Terror in Europe','Brazil Political Crisis','Climate Change',
            'Syrian Civil War', 'Migrant Crisis','Israel & Palestine','China Currency Devaluation','Election 2016',
            'Syrian Civil War','Migrant Crisis','Syrian Civil War','Supreme Court']

In [497]:
expanded = zip(refined_list, refined_tags)
as_is = zip(good_clusters, good_tags)

In [498]:
as_is

[('201633_nothingbetter', 'Terror in Europe'),
 ('2016413_nothingbetter', 'Brazil Political Crisis'),
 ('2014610_nothingbetter', 'Climate Change'),
 ('20161215_nothingbetter', 'Syrian Civil War'),
 ('201634_nothingbetter', 'Migrant Crisis'),
 ('20151011_nothingbetter', 'Israel & Palestine'),
 ('201585_nothingbetter', 'China Currency Devaluation'),
 ('201644_nothingbetter', 'Election 2016'),
 ('201690_nothingbetter', 'Syrian Civil War'),
 ('201593_nothingbetter', 'Migrant Crisis'),
 ('2015103_nothingbetter', 'Syrian Civil War'),
 ('201623_nothingbetter', 'Supreme Court')]

In [499]:
expanded

[('2015713_0.869490242596_1', 'Election 2016'),
 ('201483_0.894607405407_0', 'Israel & Palestine'),
 ('2015108_0.862314958445_3', 'Election 2016'),
 ('2012310_0.863418023586_5', 'Supreme Court'),
 ('20151117_0.868103909337_4', 'Israel & Palestine'),
 ('2014914_0.867348746541_3', 'Climate Change'),
 ('201369_0.876800446215_4', 'Egyptian Revolution'),
 ('201550_0.867353796742_6', 'Brexit'),
 ('201589_0.874460711186_1', 'Election 2016'),
 ('2015921_0.889312561455_1', 'Apple Inc.'),
 ('20141111_0.863949060139_5', 'Russian Foreign Policy'),
 ('2015620_0.857974306639_1', 'Yemeni Civil War'),
 ('2015123_0.86499726492_1', 'Climate Change'),
 ('20151110_0.851589584328_5', 'Election 2016'),
 ('2013105_0.889653955894_1', 'Climate Change'),
 ('20151117_0.872713338606_0', 'Israel & Palestine'),
 ('201274_0.855487793009_10', 'Syrian Civil War'),
 ('2016912_0.872761681001_0', 'Philippines Drug War'),
 ('2015615_0.898982075108_0', 'FIFA Corruption'),
 ('2014914_0.870578466479_2', 'Climate Change'),
 (

In [500]:
clusters_df = pd.DataFrame(columns=['url','date','text','title','length','tokens','lemmas','source','datetime','year','month','tshort','cluster'])
for item in as_is:
    root = 'good_clusters/'
    file_loc = root + item[0]
    print file_loc
    temp_df = pd.read_json(file_loc, orient='values')
    temp_df.columns = ['url','date','text','title','length','tokens','lemmas','source','datetime','year','month','tshort','cluster']
    temp_df['cluster'] = item[1]
    clusters_df = pd.concat([clusters_df,temp_df])

good_clusters/201633_nothingbetter
good_clusters/2016413_nothingbetter
good_clusters/2014610_nothingbetter
good_clusters/20161215_nothingbetter
good_clusters/201634_nothingbetter
good_clusters/20151011_nothingbetter
good_clusters/201585_nothingbetter
good_clusters/201644_nothingbetter
good_clusters/201690_nothingbetter
good_clusters/201593_nothingbetter
good_clusters/2015103_nothingbetter
good_clusters/201623_nothingbetter


In [501]:
for item in expanded:
    root = 'refined/'
    file_loc = root + item[0]
    print file_loc
    temp_df = pd.read_json(file_loc, orient='values')
    temp_df.columns = ['url','date','text','title','length','tokens','lemmas','source','datetime','year','month','tshort','filter','cluster']
    temp_df = temp_df.drop(['tshort','filter'], axis=1)
    temp_df['cluster'] = item[1]
    clusters_df = pd.concat([clusters_df,temp_df])

refined/2015713_0.869490242596_1
refined/201483_0.894607405407_0
refined/2015108_0.862314958445_3
refined/2012310_0.863418023586_5
refined/20151117_0.868103909337_4
refined/2014914_0.867348746541_3
refined/201369_0.876800446215_4
refined/201550_0.867353796742_6
refined/201589_0.874460711186_1
refined/2015921_0.889312561455_1
refined/20141111_0.863949060139_5
refined/2015620_0.857974306639_1
refined/2015123_0.86499726492_1
refined/20151110_0.851589584328_5
refined/2013105_0.889653955894_1
refined/20151117_0.872713338606_0
refined/201274_0.855487793009_10
refined/2016912_0.872761681001_0
refined/2015615_0.898982075108_0
refined/2014914_0.870578466479_2
refined/2016111_0.864964236906_8
refined/2016621_0.880303583745_0
refined/2015915_0.853669332983_0
refined/201274_0.869142411295_11
refined/2014816_0.888438440359_2
refined/2014122_0.863600010955_7
refined/2015519_0.854170619314_0
refined/201615_0.894225562833_1
refined/2013916_0.865737531586_7
refined/2015519_0.858457904117_3
refined/2016

In [502]:
clusters_df = clusters_df.reset_index()

In [503]:
clusters_df = clusters_df.drop('index', axis=1)

In [525]:
clusters_df.head()

,cluster,date,datetime,lemmas,length,month,source,text,title,tokens,tshort,url,year
0,Terror in Europe,26 Mar 2016 10:42 GMT,03/26/2016,"Brussels , Belgium - A popular teacher who work at an Islamic school in Brussels be among those kill in the Brussels attack . Teachers and student have be mourn the loss of Loubna Lafquiri , a young mother of three , after the confirmation of her...",1785.0,3.0,aljazeera,"Brussels, Belgium - A popular teacher who worked at an Islamic school in Brussels was among those killed in the Brussels attacks. Teachers and students have been mourning the loss of Loubna Lafquiri, a young mother of three, after the confirmati...",Muslim teacher among dead in Brussels attack,"[Brussels, ,, Belgium, -, A, popular, teacher, who, worked, at, an, Islamic, school, in, Brussels, was, among, those, killed, in, the, Brussels, attacks, ., Teachers, and, students, have, been, mourning, the, loss, of, Loubna, Lafquiri, ,, a, you...","[brussels, ,, belgium, -, a, popular, teacher, who, worked, at, an, islamic, school, in, brussels, was, among, those, killed, in, the, brussels, attacks, ., teachers, and, students, have, been, mourning, the, loss, of, loubna, lafquiri, ,, a, you...",http://www.aljazeera.com/news/2016/03/muslim-teacher-dead-brussels-attack-160326084605003.html,2016.0
1,Terror in Europe,2016-03-15T11:27:39-04:00,03/15/2016,"PARIS A police counterterrorism raid in Brussels link to the Paris attack turn into a chaotic shootout on Tuesday , leave one suspect dead clutch an assault rifle , at least one other on the run , four officer wound and an entire neighborhood in ...",4035.0,3.0,nytimes,"PARIS — A police counterterrorism raid in Brussels linked to the Paris attacks turned into a chaotic shootout on Tuesday, leaving one suspect dead clutching an assault rifle, at least one other on the run, four officers wounded and an entire ne...",Brussels Raid Linked to Paris Attacks Erupts in Fatal Shootout,"[PARIS, A, police, counterterrorism, raid, in, Brussels, linked, to, the, Paris, attacks, turned, into, a, chaotic, shootout, on, Tuesday, ,, leaving, one, suspect, dead, clutching, an, assault, rifle, ,, at, least, one, other, on, the, run, ,, f...","[paris, —, a, police, counterterrorism, raid, in, brussels, linked, to, the, paris, attacks, turned, into, a, chaotic, shootout, on, tuesday, ,, leaving, one, suspect, dead, clutching, an, assault, rifle, ,, at, least, one, other, on, the, run, ,...",http://www.nytimes.com/2016/03/16/world/europe/belgium-police-counterterrorism-raid.html,2016.0
2,Terror in Europe,2016-03-16T07:45:40-04:00,03/16/2016,"PARIS The man who be kill during a counterterrorism raid be an Algerian with potential link to radical Islam , Belgian prosecutor say on Wednesday . Two other suspect from Tuesday 's raid , which leave four police officer wound , remain on the ru...",3963.0,3.0,nytimes,"PARIS — The man who was killed during a counterterrorism raid was an Algerian with potential links to radical Islam, Belgian prosecutors said on Wednesday. Two other suspects from Tuesday’s raid, which left four police officers wounded, remain ...",Belgium Says Algerian Killed in Raid May Have Had Links to Radical Islam,"[PARIS, The, man, who, was, killed, during, a, counterterrorism, raid, was, an, Algerian, with, potential, links, to, radical, Islam, ,, Belgian, prosecutors, said, on, Wednesday, ., Two, other, suspects, from, Tuesday, 's, raid, ,, which, left, ...","[paris, —, the, man, who, was, killed, during, a, counterterrorism, raid, was, an, algerian, with, potential, links, to, radical, islam, ,, belgian, prosecutors, said, on, wednesday, ., two, other, suspects, from, tuesday, 's, raid, ,, which, lef...",http://www.nytimes.com/2016/03/17/world/europe/belgium-terrorism-suspect.html,2016.0
3,Terror in Europe,2016-03-18T10:03:58-04:00,03/18/2016,"BRUSSELS He travel back to Belgium unrecognized and unchallenged the day after the Nov . 13 terrorist attack in Paris . His fingerprint be find in two ap

In [ ]:
def get_people(x):
        x = tokenize(x)
        entities = ner.extract_entities(x)
        ent_list = []
        for e in entities:
            if (e[1]=='PERSON') & (len(e[0])>1) & (e[2] > 1) :
                
                rangee = e[0]
                entity_text = " ".join(x[i] for i in rangee)
#                 print e[2],' ',entity_text
                entity_text.decode("utf-8").encode("ascii","ignore")
                ent_list.append(entity_text)
        return(list(set(ent_list)))

def get_locations(x):
        x = tokenize(x)
        entities = ner.extract_entities(x)
        ent_list = []
        for e in entities:
            if (e[1]=='LOCATION') & (e[2] >1) :
                
                rangee = e[0]
                entity_text = " ".join(x[i] for i in rangee)
#                 print e[2],' ',entity_text
                entity_text.decode("utf-8").encode("ascii","ignore")
                ent_list.append(entity_text)
        return(list(set(ent_list)))

def get_orgs(x):
        x = tokenize(x)
        entities = ner.extract_entities(x)
        ent_list = []
        for e in entities:
            if (e[1]=='ORGANIZATION') & (e[2] >1) :
                
                rangee = e[0]
                entity_text = " ".join(x[i] for i in rangee)
#                 print e[2],' ',entity_text
                entity_text.decode("utf-8").encode("ascii","ignore")
                ent_list.append(entity_text)
        return(list(set(ent_list)))

In [ ]:
clusters_df['organizations'] = clusters_df['text'].apply(lambda x: get_orgs(x))

In [ ]:
clusters_df['people'] = clusters_df['text'].apply(lambda x: get_people(x))

In [526]:
clusters_df['url'] = clusters_df['url'].apply(lambda x: x.split('?')[0])
clusters_df['url'] = clusters_df['url'].apply(lambda x: x.split('/print')[0])

In [527]:
clusters_df['datetime'] = clusters_df['date'].apply(lambda x: date_conv(x))
clusters_df['datetime'] = clusters_df['datetime'].apply(lambda x: x.strftime(format='%m/%d/%Y'))

In [528]:
clusters_df['people'] = clusters_df['people'].apply(lambda x: '+'.join(x))
clusters_df['locations'] = clusters_df['locations'].apply(lambda x: '+'.join(x))

KeyError: 'people'

In [534]:
keshif_df = copy(clusters_df.drop(['text','tokens','lemmas','date','tshort'], axis=1))

In [535]:
keshif_df.head()

,cluster,datetime,length,month,source,title,url,year
0,Terror in Europe,03/26/2016,1785.0,3.0,aljazeera,Muslim teacher among dead in Brussels attack,http://www.aljazeera.com/news/2016/03/muslim-teacher-dead-brussels-attack-160326084605003.html,2016.0
1,Terror in Europe,03/15/2016,4035.0,3.0,nytimes,Brussels Raid Linked to Paris Attacks Erupts in Fatal Shootout,http://www.nytimes.com/2016/03/16/world/europe/belgium-police-counterterrorism-raid.html,2016.0
2,Terror in Europe,03/16/2016,3963.0,3.0,nytimes,Belgium Says Algerian Killed in Raid May Have Had Links to Radical Islam,http://www.nytimes.com/2016/03/17/world/europe/belgium-terrorism-suspect.html,2016.0
3,Terror in Europe,03/18/2016,7923.0,3.0,nytimes,"Salah Abdeslam, Suspect in Paris Attacks, Is Captured in Brussels",http://www.nytimes.com/2016/03/19/world/europe/salah-abdeslam-belgium-apartment.html,2016.0
4,Terror in Europe,03/18/2016,7923.0,3.0,nytimes,"Salah Abdeslam, Suspect in Paris Attacks, Is Captured in Brussels",http://www.nytimes.com/2016/03/19/world/europe/salah-abdeslam-belgium-apartment.html,2016.0


In [536]:
keshif_df = keshif_df.drop_duplicates('url')

In [544]:
keshif_df['title'] = keshif_df['title'].apply(lambda x: x.replace('\u2019',''))
keshif_df['title'] = keshif_df['title'].apply(lambda x: x.encode('utf-8').decode('ascii','ignore'))

In [545]:
keshif_df

,cluster,datetime,length,month,source,title,url,year
0,Terror in Europe,03/26/2016,1785.0,3.0,aljazeera,Muslim teacher among dead in Brussels attack,http://www.aljazeera.com/news/2016/03/muslim-teacher-dead-brussels-attack-160326084605003.html,2016.0
1,Terror in Europe,03/15/2016,4035.0,3.0,nytimes,Brussels Raid Linked to Paris Attacks Erupts in Fatal Shootout,http://www.nytimes.com/2016/03/16/world/europe/belgium-police-counterterrorism-raid.html,2016.0
2,Terror in Europe,03/16/2016,3963.0,3.0,nytimes,Belgium Says Algerian Killed in Raid May Have Had Links to Radical Islam,http://www.nytimes.com/2016/03/17/world/europe/belgium-terrorism-suspect.html,2016.0
3,Terror in Europe,03/18/2016,7923.0,3.0,nytimes,"Salah Abdeslam, Suspect in Paris Attacks, Is Captured in Brussels",http://www.nytimes.com/2016/03/19/world/europe/salah-abdeslam-belgium-apartment.html,2016.0
5,Terror in Europe,03/19/2016,21793.0,3.0,nytimes,A View of ISISs Evolution in New Details of Paris Attacks,http://www.nytimes.com/2016/03/20/world/europe/a-view-of-isiss-evolution-in-new-details-of-paris-attacks.html,2016.0
6,Terror in Europe,03/19/2016,3313.0,3.0,nytimes,"Paris Suspect Claims He Backed Out of Stadium Bombing, Prosecutor Says",http://www.nytimes.com/2016/03/20/world/europe/paris-terror-attacks-suspect-belgium.html,2016.0
7,Terror in Europe,03/21/2016,9428.0,3.0,nytimes,"The Arrest of Salah Abdeslam, a Paris Suspect, Ends Manhunt, Not Questions",http://www.nytimes.com/2016/03/22/world/europe/arrest-salah-abdeslam-paris-suspect.html,2016.0
8,Terror in Europe,03/21/2016,3750.0,3.0,nytimes,Belgian Police Name Man Suspected of Being Salah Abdeslams Accomplice,http://www.nytimes.com/2016/03/22/world/europe/paris-attacks-accomplice-najim-laachraoui.html,2016.0
9,Terror in Europe,03/22/2016,3783.0,3.0,nytimes,Standing With Brussels Against Terrorism and Fear,http://www.nytimes.com/2016/03/23/opinion/standing-with-brussels-against-terrorism-and-fear.html,2016.0
10,Terror in Europe,03/22/2016,6546.0,3.0,nytimes,Porous Borders and Calcified Security Aid Islamic State in Europe,http://www.nytimes.com/2016/03/23/us/politics/porous-borders-and-calcified-security-aid-islamic-state-in-europe.html,2016.0


In [547]:
keshif_df.to_csv('/home/ec2-user/viz_w_titles.csv')